## Installation

In [1]:
! pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 4.8 MB/s eta 0:00:00


## API

In [2]:
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

In [3]:
from together import Together

class ChatBot:
    """
    A simple ChatBot class to interact with a Together LLM model.

    Attributes:
        api_key (str): The API key used to authenticate with the Together API.
        client (Together): A Together client for making requests.
        history (list[dict]): A list of dictionaries representing the conversation history.
    """

    def __init__(self, api_key: str) -> None:
        """
        Initializes the ChatBot with a given API key and an empty conversation history.
        Also creates a Together client instance for making requests.

        Args:
            api_key (str): The API key for Together.
        """
        self.api_key: str = api_key
        self.client: Together = Together(api_key=self.api_key)
        self.history: list[dict] = []

    def append_history(self, role: str, content: str) -> None:
        """
        Appends a new message entry to the conversation history.

        Args:
            role (str): The role of the message sender, e.g., "user" or "assistant".
            content (str): The message content to be appended.
        """
        self.history.append({"role": role, "content": content})

    def invoke_api(
        self,
        model: str = "deepseek-ai/DeepSeek-V3",
        max_tokens: int = 1024,
        temperature: float = 0.7,
        top_p: float = 0.7,
        top_k: int = 50,
        repetition_penalty: float = 1.0,
        stop: list[str] = ["<｜end▁of▁sentence｜>"]
    ) -> str:
        """
        Invokes the Together chat API using the stored conversation history.

        Args:
            model (str, optional): The name of the Together model to use. Defaults to "deepseek-ai/DeepSeek-V3".
            max_tokens (int, optional): The maximum number of tokens in the response. Defaults to 1024.
            temperature (float, optional): The sampling temperature. Defaults to 0.7.
            top_p (float, optional): The top_p sampling parameter. Defaults to 0.7.
            top_k (int, optional): The top_k sampling parameter. Defaults to 50.
            repetition_penalty (float, optional): The repetition penalty parameter. Defaults to 1.0.
            stop (list[str], optional): A list of stop tokens. Defaults to ["<｜end▁of▁sentence｜>"].

        Returns:
            str: The collapsed string response from the API.
        """
        response = self.client.chat.completions.create(
            model=model,
            messages=self.history,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            stop=stop,
            stream=True
        )
        answer: str = self.collapse_response(response)
        return answer

    def collapse_response(self, response) -> str:
        """
        Collapses a streaming response from the Together API into a single string.

        Args:
            response: The streaming response object from the Together API.

        Returns:
            str: A single string containing the concatenated content from each token in the response.
        """
        answer: str = ""
        for token in response:
            if hasattr(token, "choices"):
                try:
                    answer += token.choices[0].delta.content
                except:
                    pass
        return answer

    def show_history(self) -> None:
        """
        Prints the entire conversation history.
        """
        print(self.history)


### Inference

In [4]:
# # Instantiate the ChatBot
# bot = ChatBot(api_key=TOGETHER_API_KEY)

# bot.history = [{"role": "assistant", "content": "You always provide reasoning. Your answer starts from <think>xxx</think> and <response>."}]

# # Provided by data
# current_question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
# current_answer = "72"
# augmented_content = f"Provide reasoning how to answer question: {current_question} and to arrive with answer: {current_answer}"
# print(augmented_content)

# # Append augmented content to history
# bot.append_history(role="user", content=augmented_content)
# bot.invoke_api()

In [5]:
# import os

# # Replace with your actual Together API key
# # TOGETHER_API_KEY = "YOUR_TOGETHER_API_KEY"

# # Instantiate the ChatBot
# bot = ChatBot(api_key=TOGETHER_API_KEY)
# print("Welcome to the ChatBot! Type 'exit' or 'quit' to end the conversation.\n")

# while True:
#     try:
#         # Get user input
#         user_input = input("🧑‍💻 You: ")

#         # Check for exit condition
#         if user_input.strip().lower() in ["exit", "quit"]:
#             print("👋 Ending the conversation. Goodbye!")
#             break

#         # Append user message to history
#         bot.append_history(role="user", content=user_input)

#         # Invoke the API to get the assistant's response
#         assistant_response = bot.invoke_api()

#         # Append assistant response to history
#         bot.append_history(role="assistant", content=assistant_response)

#         # Display the assistant's response with emoji
#         print(f"🤖 Assistant: {assistant_response}\n")

#     except KeyboardInterrupt:
#         print("\n👋 Conversation interrupted. Goodbye!")
#         break
#     except Exception as e:
#         print(f"❌ An error occurred: {e}")


## Installation of Dataset

In [6]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00


In [7]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

In [ ]:
# Load the full dataset
data = load_dataset('openai/gsm8k', 'main')

# Select only the first 100 samples
data = data['train'].select(range(2000))

# Print dataset summary
print(data)

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

In [ ]:
data

## Prep for Augment Data

In [ ]:
import time
from datasets import Dataset
from tqdm import tqdm  # For progress tracking

# Function to augment an answer using the ChatBot
def augment_answer(example: dict) -> dict:
    """
    Takes a dictionary containing a 'question' and 'answer',
    creates a new ChatBot instance with a clean history,
    calls the ChatBot, and replaces the answer with the augmented response.

    Args:
    example (dict): A dictionary with keys 'question' and 'answer'.

    Returns:
    dict: The updated dictionary with an augmented 'answer'.
    """
    # Instantiate a new ChatBot (clean slate)
    bot = ChatBot(api_key=TOGETHER_API_KEY)

    # Set up initial chatbot history
    bot.history = [
        {"role": "assistant", "content": "You always provide reasoning. Your answer starts from <think>xxx</think> and <response>."}
    ]

    current_question = example["question"]
    current_answer = example["answer"]

    # Construct the augmented content
    augmented_content = f"Provide reasoning how to answer question: {current_question} and to arrive with answer: {current_answer}"

    # Append question to chatbot history
    bot.append_history(role="user", content=augmented_content)

    attempts = 0
    while attempts < 3:
        try:
            # Call API and get response
            response = bot.invoke_api()
            example["answer"] = response  # Assign bot output to 'answer'
            return example
        except RateLimitError as e:
            if attempts < 2:  # try up to 3 times
                time.sleep(5)  # wait for 5 seconds before retrying
            else:
                example["answer"] = "NULL"  # set response to NULL after 3 failed attempts
                return example
        attempts += 1


In [ ]:
import os

# Get the number of CPU cores
num_cpus = os.cpu_count()

# Print the result
print(f"Number of CPUs: {num_cpus}")

In [ ]:
import tensorflow as tf

# Get the number of available GPUs
num_gpus = len(tf.config.list_physical_devices('GPU'))

# Print the result
print(f"Number of GPUs: {num_gpus}")

In [ ]:
%%time

# Apply the function to each row in the dataset
data = data.map(augment_answer, num_proc=8)  # Parallel processing for speed

In [ ]:
# Save the updated dataset (optional)
data.save_to_disk("augmented_gsm8k_2k")

In [ ]:
# data = data.map(lambda x: { # type: ignore
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': extract_hash_answer(x['answer'])
#     }) # type: ignore

In [ ]:
from datasets import load_from_disk, DatasetDict

# Load dataset from the saved folder
aug_data = load_from_disk("augmented_gsm8k_2k")

# Convert it into a DatasetDict format
dataset_dict = DatasetDict({"train": aug_data})

# Save again in correct structure
dataset_dict.save_to_disk("augmented_gsm8k_2k")

# Define user and repo details
user_name = "eagle0504"
repo_name = "openai-gsm8k-augmented-using-together-ai-deepseek-v3-train-enhanced-2k"  # Change this to your desired repository name

# Push to Hugging Face
dataset_dict.push_to_hub(f"{user_name}/{repo_name}")

# Print the URL of the pushed dataset
print(f"Dataset pushed to: https://huggingface.co/datasets/{user_name}/{repo_name}")